In [1]:
import os
import warnings

import netCDF4 as nc
import numpy as np
import xarray as xr
import pandas as pd
import multiprocessing as mp
import matplotlib.pyplot as plt

from glob import glob

warnings.filterwarnings('ignore')
os.environ['OMP_NUM_THREADS'] = '1'

## Pull in GFSDS output

In [ ]:
flist = sorted(glob('./gfsds/*.nc'))

gfsds = xr.open_mfdataset(flist)
gfsds['lead_time'] = ((gfsds.time - gfsds.time[0])/(3.6e12)).astype(int)

gfsds

## Ensure tally method functions properly

In [ ]:
xqpf = gfsds['dqpf'].sum(dim='time')
xqpf_id = np.where(xqpf == xqpf.max())
xqpf_sample = gfsds.isel(lon=xqpf_id[1], lat=xqpf_id[0]).load()
xqpf_sample = xqpf_sample.isel(lat=0, lon=0)

In [ ]:
apcp_24_test = []
for lead in np.arange(24, 84+1, 12):
    tidx = np.where(gfsds['lead_time'] == lead)[0][0]
    apcp_24 = xqpf_sample.isel(time=slice(tidx-7, tidx+1))['dqpf'].sum(dim='time')
    apcp_24_test.append([lead, apcp_24.values])
apcp_24_test = np.array(apcp_24_test)

In [ ]:
fig, ax = plt.subplots(1, facecolor='w')

ax.plot(xqpf_sample['lead_time'], xqpf_sample['dqpf'], 
        color='k', linestyle='--', label='3h APCP')
ax.scatter(apcp_24_test[:, 0], apcp_24_test[:, 1], 
           marker='x', color='k', s=200, linewidth=1.5, label='24h APCP')

ax.set_title('sample APCP')
ax.legend()
ax.grid()
plt.show()

## 2d Lat Lon Needed, use Meshgrid

In [ ]:
lat, lon = gfsds['lat'], gfsds['lon']
lon2d, lat2d = np.meshgrid(lon, lat)

## Create NetCDF Files for WPC MODE 
#### Tally APCP, ASNOW, mean SLR for 24 hours/every 12 hours

In [ ]:
use_lambert_sample = False
accum24_sec = str(60 * 60 * 24)

In [ ]:
for lead in np.arange(24, 84+1, 12):
    
    tidx = np.where(gfsds['lead_time'] == lead)[0][0]
    tslice = slice(tidx-7, tidx+1)
        
    itime = pd.to_datetime(gfsds.time.values[0])
    vtime = pd.to_datetime(gfsds.isel(time=tslice).time.values[-1])

    apcp_24 = gfsds.isel(time=tslice)['dqpf'].sum(dim='time')
    asnow_24 = gfsds.isel(time=tslice)['dqsf'].sum(dim='time')
    
    slr_24 = xr.where(asnow_24 > 0, gfsds.isel(time=slice(tidx-7, tidx+1))['slr'], np.nan)
    slr_24 = slr_24.mean(dim='time', skipna=True)
    slr_24 = xr.where(~np.isnan(slr_24), slr_24, 0.)

    if use_lambert_sample:
        ncfilename = './mode/uutah_downscaled_gfs.%s_f%03d_v%s.nc'%(
            itime.strftime('%Y%m%d_%H%M%S'), lead, vtime.strftime('%Y%m%d_%H%M%S'))
    else:
        ncfilename = './mode/uutah_downscaled_gfs.no_proj.%s_f%03d_v%s.nc'%(
            itime.strftime('%Y%m%d_%H%M%S'), lead, vtime.strftime('%Y%m%d_%H%M%S'))
        
    try:
        os.remove(ncfilename)
    except:
        pass
        
    ncfile = nc.Dataset(ncfilename, 'w', format='NETCDF4')

    # Global Attributes
    ncfile.MET_version = "V6.0";
    ncfile.MET_tool = "pcp_combine";
    
    if use_lambert_sample:
        ncfile.Projection = "Lambert Conformal";
        ncfile.scale_lat_1 = "38.500000";
        ncfile.scale_lat_2 = "38.500000";
        ncfile.lat_pin = "21.138000";
        ncfile.lon_pin = "-122.720000";
        ncfile.x_pin = "0.000000";
        ncfile.y_pin = "0.000000";
        ncfile.lon_orient = "-97.500000";
        ncfile.d_km = "3.000000";
        ncfile.r_km = "6371.200000";
    else:
        ncfile.Projection = "LatLon";
        ncfile.longitude_of_prime_meridian = 0.0 ;
        ncfile.semi_major_axis = 6378137.0 ;
        ncfile.inverse_flattening = 298.257223563 ;
    
    ncfile.nx = str(len(lon))
    ncfile.ny = str(len(lat))

    # Lat Lon dimensions and data
    ncfile.createDimension('lon', len(lon))
    ncfile.createDimension('lat', len(lat))

    lon_nc = ncfile.createVariable('lon', 'f4', ('lat', 'lon'))
    lon_nc.long_name = 'longitude'
    lon_nc.units = 'degrees_east'
    lon_nc.standard_name = 'longitude'
    lon_nc._CoordinateAxisType = 'Lon'

    lat_nc = ncfile.createVariable('lat', 'f4', ('lat', 'lon'))
    lat_nc.long_name = 'latitude'
    lat_nc.units = 'degrees_north'
    lat_nc.standard_name = 'latitude'
    lat_nc._CoordinateAxisType = 'Lat'

    lon_nc[:] = lon2d
    lat_nc[:] = lat2d

    # Write variable data
    apcp_24_nc = ncfile.createVariable('APCP_R1', 'f4', ('lat', 'lon'), fill_value=-9999.0)
    apcp_24_nc.long_name = 'Total Precipitation'
    apcp_24_nc.level = 'R1'
    apcp_24_nc.units = 'inches'
    apcp_24_nc.init_time = itime.strftime('%Y%m%d_%H%M%S')
    apcp_24_nc.init_time_ut = str(itime.timestamp())
    apcp_24_nc.valid_time = vtime.strftime('%Y%m%d_%H%M%S')
    apcp_24_nc.valid_time_ut = str(vtime.timestamp())
    apcp_24_nc.accum_time_sec = accum24_sec

    asnow_24_nc = ncfile.createVariable('ASNOW_R1', 'f4', ('lat', 'lon'), fill_value=-9999.0)
    asnow_24_nc.long_name = 'Total Snow'
    asnow_24_nc.level = 'R1'
    asnow_24_nc.units = 'inches'
    asnow_24_nc.init_time = itime.strftime('%Y%m%d_%H%M%S')
    asnow_24_nc.init_time_ut = str(itime.timestamp())
    asnow_24_nc.valid_time = vtime.strftime('%Y%m%d_%H%M%S')
    asnow_24_nc.valid_time_ut = str(vtime.timestamp())
    asnow_24_nc.accum_time_sec = accum24_sec

    slr_24_nc = ncfile.createVariable('SLR_R1', 'f4', ('lat', 'lon'), fill_value=-9999.0)
    slr_24_nc.long_name = 'Snow Liquid Ratio'
    slr_24_nc.level = 'R1'
    slr_24_nc.units = 'ratio'
    slr_24_nc.init_time = itime.strftime('%Y%m%d_%H%M%S')
    slr_24_nc.init_time_ut = str(itime.timestamp())
    slr_24_nc.valid_time = vtime.strftime('%Y%m%d_%H%M%S')
    slr_24_nc.valid_time_ut = str(vtime.timestamp())
    slr_24_nc.accum_time_sec = accum24_sec
    
    mm_in = 1/25.4
    apcp_24_nc[:] = apcp_24.values * mm_in
    asnow_24_nc[:] = asnow_24.values * mm_in
    slr_24_nc[:] = slr_24.values
    
    print('lead %d ncfile %s written'%(lead, ncfilename))
    ncfile.close()